In [1]:
import numpy as np
import pickle
import os

from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import Bunch
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.io import imread
from skimage.transform import resize
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D
import os.path
import fnmatch
import itertools
import functools
from keras.models import Model

Using TensorFlow backend.


In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
train_data_dir = 'data/train'
validation_data_dir = 'data/val'

In [4]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [5]:
EPOCHS = 100
INIT_LR = 1e-3
BS = 8
image_size = 0
width=299
height=299
depth=3

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(height, width),
    batch_size=BS,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(height, width),
    batch_size=BS,
    class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(height, width),
    batch_size=BS,
    class_mode='categorical',
    shuffle = False)

Found 2800 images belonging to 7 classes.
Found 700 images belonging to 7 classes.
Found 700 images belonging to 7 classes.


In [7]:
base_model = InceptionV3(weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(7, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

In [9]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('best_model.h5', verbose=1, monitor='val_accuracy', save_best_only=True, mode='auto') 

In [11]:
history = model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.n // BS,
    validation_steps=validation_generator.n // BS,
    epochs=EPOCHS, verbose=1,
    callbacks=[checkpoint],
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
350/350 [==============================] - ETA: 0s - loss: 1.9480 - accuracy: 0.2714
Epoch 00001: val_accuracy improved from -inf to 0.30891, saving model to best_model.h5
350/350 [==============================] - 53s 152ms/step - loss: 1.9480 - accuracy: 0.2714 - val_loss: 2.4118 - val_accuracy: 0.3089
Epoch 2/100
350/350 [==============================] - ETA: 0s - loss: 1.5727 - accuracy: 0.4100
Epoch 00002: val_accuracy improved from 0.30891 to 0.43247, saving model to best_model.h5
350/350 [==============================] - 52s 149ms/step - loss: 1.5727 - accuracy: 0.4100 - val_loss: 1.5751 - val_accuracy: 0.4325
Epoch 3/100
350/350 [==============================] - ETA: 0s - loss: 1.4086 - accuracy: 0.4779
Epoch 00003: val_accuracy did not improve from 0.43247
350/350 [==============================] - 51s 146ms/step - loss: 1.4086 - accuracy: 0.4779 - val_loss: 2.1938 - val_accuracy: 0.3520

Epoch 28/100
350/350 [==============================] - ETA: 0s - loss: 0.1962 - accuracy: 0.9471
Epoch 00028: val_accuracy improved from 0.67241 to 0.67529, saving model to best_model.h5
350/350 [==============================] - 52s 150ms/step - loss: 0.1962 - accuracy: 0.9471 - val_loss: 2.3804 - val_accuracy: 0.6753
Epoch 29/100
350/350 [==============================] - ETA: 0s - loss: 0.1969 - accuracy: 0.9468
Epoch 00029: val_accuracy did not improve from 0.67529
350/350 [==============================] - 53s 150ms/step - loss: 0.1969 - accuracy: 0.9468 - val_loss: 6.2559 - val_accuracy: 0.4511
Epoch 30/100
350/350 [==============================] - ETA: 0s - loss: 0.1608 - accuracy: 0.9571
Epoch 00030: val_accuracy did not improve from 0.67529
350/350 [==============================] - 53s 150ms/step - loss: 0.1608 - accuracy: 0.9571 - val_loss: 2.4778 - val_accuracy: 0.6566
Epoch 31/100
350/350 [==============================] - ETA: 0s - loss: 0.2188 - accuracy: 0.9518
Epoch 

350/350 [==============================] - ETA: 0s - loss: 0.1167 - accuracy: 0.9771
Epoch 00056: val_accuracy did not improve from 0.69971
350/350 [==============================] - 52s 149ms/step - loss: 0.1167 - accuracy: 0.9771 - val_loss: 4.6898 - val_accuracy: 0.6681
Epoch 57/100
350/350 [==============================] - ETA: 0s - loss: 0.1555 - accuracy: 0.9725
Epoch 00057: val_accuracy did not improve from 0.69971
350/350 [==============================] - 52s 149ms/step - loss: 0.1555 - accuracy: 0.9725 - val_loss: 4.4868 - val_accuracy: 0.6351
Epoch 58/100
350/350 [==============================] - ETA: 0s - loss: 0.1606 - accuracy: 0.9743
Epoch 00058: val_accuracy did not improve from 0.69971
350/350 [==============================] - 52s 149ms/step - loss: 0.1606 - accuracy: 0.9743 - val_loss: 4.6065 - val_accuracy: 0.6437
Epoch 59/100
350/350 [==============================] - ETA: 0s - loss: 0.1512 - accuracy: 0.9739
Epoch 00059: val_accuracy improved from 0.69971 to 0.7

350/350 [==============================] - ETA: 0s - loss: 0.0901 - accuracy: 0.9800
Epoch 00084: val_accuracy did not improve from 0.71264
350/350 [==============================] - 52s 148ms/step - loss: 0.0901 - accuracy: 0.9800 - val_loss: 6.3032 - val_accuracy: 0.6566
Epoch 85/100
350/350 [==============================] - ETA: 0s - loss: 0.1234 - accuracy: 0.9793
Epoch 00085: val_accuracy did not improve from 0.71264
350/350 [==============================] - 53s 150ms/step - loss: 0.1234 - accuracy: 0.9793 - val_loss: 5.8311 - val_accuracy: 0.6695
Epoch 86/100
350/350 [==============================] - ETA: 0s - loss: 0.0859 - accuracy: 0.9818
Epoch 00086: val_accuracy did not improve from 0.71264
350/350 [==============================] - 53s 150ms/step - loss: 0.0859 - accuracy: 0.9818 - val_loss: 4.4994 - val_accuracy: 0.6595
Epoch 87/100
350/350 [==============================] - ETA: 0s - loss: 0.0668 - accuracy: 0.9854
Epoch 00087: val_accuracy did not improve from 0.71264

In [12]:
print("[INFO] Calculating model accuracy")
model.load_weights('best_model.h5')
scores = model.evaluate_generator(generator=test_generator, steps=test_generator.n // BS +1)
print(f"Test Accuracy: {scores[1]*100}")

[INFO] Calculating model accuracy
Instructions for updating:
Please use Model.evaluate, which supports generators.
Test Accuracy: 70.99999785423279


In [13]:
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict_generator(test_generator, steps=test_generator.n // BS + 1)

y_pred = np.argmax(Y_pred, axis=1)
print()

print('Confusion Matrix')
target_names = list(test_generator.class_indices.keys())
print(target_names)
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')

print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Instructions for updating:
Please use Model.predict, which supports generators.

Confusion Matrix
['BulBul', 'Chorui', 'Crow', 'Doel', 'Machranga', 'Shalik', 'cuckoo']
[[53  8  8  4  5 11 11]
 [ 1 64 15  4  3 12  1]
 [ 6  0 76  5  0  6  7]
 [ 1  2  3 71 14  8  1]
 [ 6 10  1  4 64 12  3]
 [ 4  5  7  3  6 64 11]
 [ 9  1  7  2  4  7 70]]
Classification Report
              precision    recall  f1-score   support

      BulBul       0.66      0.53      0.59       100
      Chorui       0.71      0.64      0.67       100
        Crow       0.65      0.76      0.70       100
        Doel       0.76      0.71      0.74       100
   Machranga       0.67      0.64      0.65       100
      Shalik       0.53      0.64      0.58       100
      cuckoo       0.67      0.70      0.69       100

    accuracy                           0.66       700
   macro avg       0.67      0.66      0.66       700
weighted avg       0.67      0.66      0.66       700

